In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

In [33]:
def compute_true_info(hessian, hessian_half,half_num_samples):
    information_true = [torch.empty_like(p) for p in hessian]
    A_list = [torch.empty_like(p) for p in hessian]
    B_list = [torch.empty_like(p) for p in hessian]
    for i,_ in enumerate(A_list):
        ratio = torch.where(torch.logical_and(hessian[i]==0, hessian_half[i]==0), 1, hessian_half[i]/hessian[i])
        A=torch.log(ratio)/half_num_samples
        B=(1/2 * (1-torch.pow(ratio,2)))/half_num_samples
        A[torch.isinf(A)]=0
        B[torch.isinf(B)]=0
        A_list[i]=torch.nan_to_num(A)
        B_list[i]=torch.nan_to_num(B)

    print("A_list", A_list)
    print("B_list", B_list)
    A_flat = torch.cat([A.flatten() for A in A_list])
    B_flat = torch.cat([B.flatten() for B in B_list])
    print("A_flat", A_flat)
    print("B_flat", B_flat)
    torch.cuda.empty_cache()
    device = torch.device("cuda")
    A_flat = A_flat.to(device)
    B_flat = B_flat.to(device)
    information_true = [torch.empty_like(p).to(device) for p in hessian]

    information_true_view = [p.view(-1) for p in information_true]
    A_list_view = [p.view(-1) for p in A_list]
    B_list_view = [p.view(-1) for p in B_list]
    print("\nstart")
    for i,_ in enumerate(information_true_view):
        for j,_ in enumerate(information_true_view[i]):
            print("A_list*A_flat", A_list_view[i][j]*A_flat)
            temp=A_list_view[i][j]*A_flat + 0.25*B_list_view[i][j]*B_flat + 0.5*A_list_view[i][j]*B_flat + 0.5*B_list_view[i][j]*A_flat
            print("temp_sum", torch.sum(temp))
            print("B_list^2", 0.5*torch.pow(B_list_view[i][j],2))
            information_true_view[i][j] = torch.sum(temp)+0.5*torch.pow(B_list_view[i][j],2)
            print("information_true_view[i][j]", information_true_view[i][j])
    return information_true


In [35]:
tesnor1=torch.tensor([2,1,3,4,5,6,7,8,9,10])
tesnor2=torch.tensor([[11,12,13,14,15,16],[17,18,19,20,21,22]])
tesnor3=torch.tensor([6,5,7,8,9,10,11,12,13,14])
tesnor4=torch.tensor([[15,16,17,18,19,20],[21,22,23,24,25,26]])
hessian=[tesnor1,tesnor2]
hessian_half=[tesnor3,tesnor4]
information_true=compute_true_info(hessian, hessian_half,1)
information_true_2=compute_true_info(hessian, hessian_half,2)
#information_true_3=compute_true_info(hessian, hessian_half,3)
for p in information_true:
    print(p)
for p in information_true_2:
    print(p)
# for p in information_true_3:
#     print(p)

A_list [tensor([1.0986, 1.6094, 0.8473, 0.6931, 0.5878, 0.5108, 0.4520, 0.4055, 0.3677,
        0.3365]), tensor([[0.3102, 0.2877, 0.2683, 0.2513, 0.2364, 0.2231],
        [0.2113, 0.2007, 0.1911, 0.1823, 0.1744, 0.1671]])]
B_list [tensor([ -4.0000, -12.0000,  -2.2222,  -1.5000,  -1.1200,  -0.8889,  -0.7347,
         -0.6250,  -0.5432,  -0.4800]), tensor([[-0.4298, -0.3889, -0.3550, -0.3265, -0.3022, -0.2812],
        [-0.2630, -0.2469, -0.2327, -0.2200, -0.2086, -0.1983]])]
A_flat tensor([1.0986, 1.6094, 0.8473, 0.6931, 0.5878, 0.5108, 0.4520, 0.4055, 0.3677,
        0.3365, 0.3102, 0.2877, 0.2683, 0.2513, 0.2364, 0.2231, 0.2113, 0.2007,
        0.1911, 0.1823, 0.1744, 0.1671])
B_flat tensor([ -4.0000, -12.0000,  -2.2222,  -1.5000,  -1.1200,  -0.8889,  -0.7347,
         -0.6250,  -0.5432,  -0.4800,  -0.4298,  -0.3889,  -0.3550,  -0.3265,
         -0.3022,  -0.2812,  -0.2630,  -0.2469,  -0.2327,  -0.2200,  -0.2086,
         -0.1983])

start
A_list*A_flat tensor([1.2069, 1.7681, 0.9309,